# Docker "Swarm Mode" Lab

Based on Mario's gist here: https://gist.github.com/l0rd/5186cc80f8f26dc7e9490abca4405830

# Requirements
- Docker 1.12
- Docker machine
- Virtualbox

# Create 3 nodes for your swarm cluster (one master and 2 slaves)

We will create 3 nodes using docker-machine/virtualbox.

#### NOTE:
You may see errors as below, and an error reported by "docker-machine ls", leave some time for the swmaster1 to settle.

In the meantime you can go ahead and create the 2 swnode's below.

In [1]:
MACHINE_DRIVER=virtualbox

CLEANUP_VMS=1
CLEANUP_VMS=0
CLEANUP_VMS=1

. ../NB_bash_functions.rc

Notebook started at Wed May 17 18:59:05 CEST 2017   [1495040345]


In [2]:
# Specific to my environment

[ -f ~/.docker.rc ] && . ~/.docker.rc

NB_continue

Setting up alias/DOCKER_HOST for 17.05.0-ce [/home/mjb/DOCKER/docker-17.05.0-ce]


In [3]:
MACHINE_DRIVER=virtualbox

#MACHINE_DRIVER=digitalocean
#MACHINE_DRIVER=azure
#MACHINE_DRIVER=google

Let's cleanup any remaining machines, if they exist already ..

In [4]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

"swmaster1" {ed891883-3fbf-4dd1-baee-338ec2582e66}
"swnode1" {a4174991-dea8-4444-bf75-03f763512558}
"swnode2" {1d030aec-82a5-4480-ae29-ab331501bdd0}

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.100:2376           v17.05.0-ce   
swnode1     -        virtualbox   Running   tcp://192.168.99.102:2376           v17.05.0-ce   
swnode2     -        virtualbox   Running   tcp://192.168.99.103:2376           v17.05.0-ce   


In [5]:
docker-machine ls --filter name=sw --format "{{.Name}}: {{.DriverName}}"

swmaster1: virtualbox
swnode1: virtualbox
swnode2: virtualbox


In [6]:
[ $CLEANUP_VMS -ne 0 ] && NB_cleanup_swarm_machines

NB_continue

docker-machine rm -f swmaster1 swnode1 swnode2
About to remove swmaster1, swnode1, swnode2
Successfully removed swmaster1
Successfully removed swnode1
Successfully removed swnode2


In [7]:
[ $CLEANUP_VMS -ne 0 ] &&  {
    VBoxManage list runningvms | grep sw

    echo
    docker-machine ls
}

NB_continue


NAME   ACTIVE   DRIVER   STATE   URL   SWARM   DOCKER   ERRORS


Now let's create our new nodes

**NOTE:** You may see some worrying messages from "*docker-machine ls*" but wait a minute or two until the node creation is complete

In [8]:
NB_create_swarm_machine_m1

Running command <docker-machine create -d virtualbox swmaster1> ...
Running pre-create checks...
Creating machine...
(swmaster1) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/swmaster1/boot2docker.iso...
(swmaster1) Creating VirtualBox VM...
(swmaster1) Creating SSH key...
(swmaster1) Starting the VM...
(swmaster1) Check network to re-create if needed...
(swmaster1) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and running!
To see how to connect your Docker Client to the Docker Engine running on this virtual machine, run: docker-machine env swmaster1
Took 67 secs [0h01m07

#### NOTE: Temporary errors from "*docker-machine ls*"

From the command-line you can see the progress of machine creation, using
    ```$ docker-machine ls```
    
You may see errors during creation of machines before ssh connectivity is established and before docker host is started.

![](images/docker-machine-errors.png)

After creation of a node you should see something like the following from "*docker-machine ls*"

In [9]:
docker-machine ls

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.104:2376           v17.05.0-ce   


We can also see the VM from virtualbox point of view (if this is the driver we are using):

In [10]:
VBoxManage list runningvms | grep sw

"swmaster1" {5f5ccbad-f087-4955-8c67-42693ae0d742}


In [11]:
NB_create_swarm_machine_1

Running command <docker-machine create -d virtualbox swnode1> ...
Running pre-create checks...
Creating machine...
(swnode1) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/swnode1/boot2docker.iso...
(swnode1) Creating VirtualBox VM...
(swnode1) Creating SSH key...
(swnode1) Starting the VM...
(swnode1) Check network to re-create if needed...
(swnode1) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and running!
To see how to connect your Docker Client to the Docker Engine running on this virtual machine, run: docker-machine env swnode1
Took 65 secs [0h01m05]


In [12]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

"swmaster1" {5f5ccbad-f087-4955-8c67-42693ae0d742}
"swnode1" {c149f283-ee67-48c6-a768-1be8ee7726ab}

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.104:2376           v17.05.0-ce   
swnode1     -        virtualbox   Running   tcp://192.168.99.105:2376           v17.05.0-ce   


In [13]:
NB_create_swarm_machine_2

Running command <docker-machine create -d virtualbox swnode2> ...
Running pre-create checks...
Creating machine...
(swnode2) Unable to get the latest Boot2Docker ISO release version:  failure getting a version tag from the Github API response (are you getting rate limited by Github?)
(swnode2) Copying /home/mjb/.docker/machine/cache/boot2docker.iso to /home/mjb/.docker/machine/machines/swnode2/boot2docker.iso...
(swnode2) Creating VirtualBox VM...
(swnode2) Creating SSH key...
(swnode2) Starting the VM...
(swnode2) Check network to re-create if needed...
(swnode2) Waiting for an IP...
Waiting for machine to be running, this may take a few minutes...
Detecting operating system of created instance...
Waiting for SSH to be available...
Detecting the provisioner...
Provisioning with boot2docker...
Copying certs to the local machine directory...
Copying certs to the remote machine...
Setting Docker configuration on the remote daemon...
Checking connection to Docker...
Docker is up and runni

In [14]:
VBoxManage list runningvms | grep sw

echo
docker-machine ls

"swmaster1" {5f5ccbad-f087-4955-8c67-42693ae0d742}
"swnode1" {c149f283-ee67-48c6-a768-1be8ee7726ab}
"swnode2" {35978041-1d3a-46ec-941e-8a9bbf4241c5}

NAME        ACTIVE   DRIVER       STATE     URL                         SWARM   DOCKER        ERRORS
swmaster1   -        virtualbox   Running   tcp://192.168.99.104:2376           v17.05.0-ce   
swnode1     -        virtualbox   Running   tcp://192.168.99.105:2376           v17.05.0-ce   
swnode2     -        virtualbox   Running   tcp://192.168.99.106:2376           v17.05.0-ce   


In [15]:
NB_check_3_machines_running

# swarm init

Now that we have 3 nodes available, we will initialize our Swarm Cluster with 1 master node.

### Directing the docker client to a particular nodes' docker daemon
**NOTE**: See that we precede all docker commands with $(docker-machine config NODE) where node is the name of the node to which we want our docker client to connect to.  This command returns the parameters to direct our client to the appropriate node.  Run alone this produces:

In [16]:
docker-machine config swmaster1

--tlsverify
--tlscacert="/home/mjb/.docker/machine/machines/swmaster1/ca.pem"
--tlscert="/home/mjb/.docker/machine/machines/swmaster1/cert.pem"
--tlskey="/home/mjb/.docker/machine/machines/swmaster1/key.pem"
-H=tcp://192.168.99.104:2376


We can also obtain these values as environment variables

In [17]:
docker-machine env swmaster1

export DOCKER_TLS_VERIFY="1"
export DOCKER_HOST="tcp://192.168.99.104:2376"
export DOCKER_CERT_PATH="/home/mjb/.docker/machine/machines/swmaster1"
export DOCKER_MACHINE_NAME="swmaster1"
# Run this command to configure your shell: 
# eval $(docker-machine env swmaster1)


Including these parameters on the docker command line will connect the client to the docker daemon running on node '*swmaster1*'.

### Networks before creation of swarm cluster
Before going further let's look at the networks on your machine.

Later, we'll see how a new network is created once the swarm cluster has been created.

In [18]:
docker $(docker-machine config swmaster1) network ls

NETWORK ID          NAME                DRIVER              SCOPE
acfce2527ff4        bridge              bridge              local
ef8503682cac        host                host                local
a28f2f5dee71        none                null                local


Now let's identify the ip address of our master node.

We can see this through config or ip commands of docker-machine as shown below.

In [19]:
docker-machine ip swmaster1

192.168.99.104


We could then provide the above ip address as parameter to --advertise-addr when initializing the swarm.

However, it is quite convenient to run the above commands embedded, as below, as arguments to the swarm init command.

docker-machine config swmaster1 provides the parameters to use when connecting to the appropriate docker engine for our machine "swmaster1".

The following command will run swarm init to generate the cluster with 'swmaster1' as the Master node.
You should see output similar to the below:

In [20]:
docker $(docker-machine config swmaster1) info | grep Swarm

Swarm: inactive


In [21]:
which docker

alias docker='/home/mjb/DOCKER/docker-17.05.0-ce/docker'
	~/DOCKER/docker-17.05.0-ce/docker


In [22]:
docker $(docker-machine config swmaster1) swarm init --advertise-addr $(docker-machine ip swmaster1)

Swarm initialized: current node (auzbedso37ndv6his2gimxopa) is now a manager.

To add a worker to this swarm, run the following command:

    docker swarm join \
    --token SWMTKN-1-01i03ariiut6svfmnscg4y8br43hss2sc8of2aavmku8j03opd-5rqu1004on1ypwdpm35f44k86 \
    192.168.99.104:2377

To add a manager to this swarm, run 'docker swarm join-token manager' and follow the instructions.



A docker info should now show "Swarm: active" as below:

In [23]:
docker $(docker-machine config swmaster1) info | grep Swarm:

Swarm: active


In [24]:
docker $(docker-machine config swmaster1) info

Containers: 0
 Running: 0
 Paused: 0
 Stopped: 0
Images: 0
Server Version: 17.05.0-ce
Storage Driver: aufs
 Root Dir: /mnt/sda1/var/lib/docker/aufs
 Backing Filesystem: extfs
 Dirs: 0
 Dirperm1 Supported: true
Logging Driver: json-file
Cgroup Driver: cgroupfs
Plugins: 
 Volume: local
 Network: bridge host macvlan null overlay
Swarm: active
 NodeID: auzbedso37ndv6his2gimxopa
 Is Manager: true
 ClusterID: rp5myydtvpe3w78fexzhml046
 Managers: 1
 Nodes: 1
 Orchestration:
  Task History Retention Limit: 5
 Raft:
  Snapshot Interval: 10000
  Number of Old Snapshots to Retain: 0
  Heartbeat Tick: 1
  Election Tick: 3
 Dispatcher:
  Heartbeat Period: 5 seconds
 CA Configuration:
  Expiry Duration: 3 months
 Node Address: 192.168.99.104
 Manager Addresses:
  192.168.99.104:2377
Runtimes: runc
Default Runtime: runc
Init Binary: docker-init
containerd version: 9048e5e50717ea4497b757314bad98ea3763c145
runc version: 9c2d8d184e5da67c95d601382adf14862e4f2228
init version: 949e6fa
Security Options:
 s

If we look at the networks we should now see new networks such as '*ingress*' an overlay network and docker_gwbridge for the swarm cluster.

In [25]:
docker $(docker-machine config swmaster1) network ls

NETWORK ID          NAME                DRIVER              SCOPE
acfce2527ff4        bridge              bridge              local
d613031008ea        docker_gwbridge     bridge              local
ef8503682cac        host                host                local
1a8ek2xs6bhv        ingress             overlay             swarm
a28f2f5dee71        none                null                local


# swarm join

Now we wish to join Master and Worker nodes to our swarm cluster, to do this we need to obtain the token generated during the "swarm init".

Let's save that token to an environment variable as follows:

In [26]:
worker_token=$(docker $(docker-machine config swmaster1) swarm join-token worker -q)

In [27]:
echo $worker_token

SWMTKN-1-01i03ariiut6svfmnscg4y8br43hss2sc8of2aavmku8j03opd-5rqu1004on1ypwdpm35f44k86


Now we can use this token to join nodes as a worker to this cluster

Note: we could also join nodes as Master, but we have only 3 nodes available.

Let's join swnode1 as a worker node

In [28]:
docker $(docker-machine config swnode1) swarm join --token $worker_token $(docker-machine ip swmaster1):2377

This node joined a swarm as a worker.


Now we can use the same token to join node swnode2 as a worker node


In [29]:
docker $(docker-machine config swnode2) swarm join --token $worker_token $(docker-machine ip swmaster1):2377

This node joined a swarm as a worker.


In [30]:
docker $(docker-machine config swmaster1) node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
63n56ml91pleofqteefs3lfyl     swnode2             Ready               Active              
auzbedso37ndv6his2gimxopa *   swmaster1           Ready               Active              Leader
elcdcst9ubchsrj1s3w9ig0i1     swnode1             Ready               Active              


# start service

First we check for any running services - there should be none in our newly initialized cluster:

In [31]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE               PORTS


Now we will create a new service based on the docker image mjbright/docker-demo

We will expose this service on port 8080


In [32]:
docker $(docker-machine config swmaster1) service create --replicas 1 --name docker-demo -p 8080:8080 mjbright/docker-demo:20

j6df1lpmiq5emi8fjlxec57zn
Since --detach=false was not specified, tasks will be created in the background.
In a future release, --detach=false will become the default.


Now we list services again and we should see our newly added docker-demo service

In [33]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE                     PORTS
j6df1lpmiq5e        docker-demo         replicated          0/1                 mjbright/docker-demo:20   *:8080->8080/tcp


In [34]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE                     PORTS
j6df1lpmiq5e        docker-demo         replicated          0/1                 mjbright/docker-demo:20   *:8080->8080/tcp


In [35]:
NB_docker_loop_until_service_started docker-demo

... waiting for 'docker-demo' replicas to start
j6df1lpmiq5e        docker-demo         replicated          1/1                 mjbright/docker-demo:20   *:8080->8080/tcp


it may take a few seconds until the service has a running container (see REPLICAS column) especially if the container image has not yet been downloaded

In [36]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE                     PORTS
j6df1lpmiq5e        docker-demo         replicated          1/1                 mjbright/docker-demo:20   *:8080->8080/tcp


... and we can look at the service as seen by the cluster:

In [37]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE           ERROR               PORTS
yqdbmwna39vb        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 3 seconds ago                       


... and we can look at the service on the individual cluster nodes.

Of course as we set replicas to 1 there is only 1 replica of the service for the moment, running on just 1 node of our cluster:

In [38]:
docker $(docker-machine config swmaster1) ps

CONTAINER ID        IMAGE                     COMMAND                  CREATED             STATUS              PORTS               NAMES
1c2b872891c1        mjbright/docker-demo:20   "/bin/docker-demo ..."   4 seconds ago       Up 4 seconds        8080/tcp            docker-demo.1.yqdbmwna39vb4j3rrhk9q853n


In [39]:
docker $(docker-machine config swnode1) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


In [40]:
docker $(docker-machine config swnode2) ps

CONTAINER ID        IMAGE               COMMAND             CREATED             STATUS              PORTS               NAMES


# Accessing the service

As we are working remotely we need to create an ssh tunnel through to the swarm cluster to access our service, exposing the port 8080 on your local machine.

We can obtain the ip address of the swarm master node as follows.

In [41]:
docker-machine ip swmaster1

192.168.99.104


Let's set an environment variable with that ip address

In [42]:
MASTERIP=$(docker-machine ip swmaster1)

Then open your web browser at the page http://<MASTERIP>:8080 (replace <MASTERIP> with the appropriate value) and you should see a lovely blue whale, as below:

![](images/docker.png)


Alternatively we can connect using a command-line client (the web server detects if it's a wget or curl request and returns ASCII text instead of an image).

In [43]:
wget -O - http://$MASTERIP:8080

--2017-05-17 19:02:58--  http://192.168.99.104:8080/
Connecting to 192.168.99.104:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/txt]
Saving to: ‘STDOUT’

-                       [<=>                 ]       0  --.-KB/s               


                                                .---------.                                          
                                               .///++++/:.                                          
                                               .///+++//:.                                          
                                               .///+++//:.                                          
                             ``````````````````.:///////:.                       `                  
                             .-///////:://+++//::///////-.                      .--.                
                             .:::///:::///+++///:::///:::.                     .:ss+-`              
          

# scale service

Now we can scale the service to 3 replicas:

In [44]:
docker $(docker-machine config swmaster1) service scale docker-demo=3

docker-demo scaled to 3


In [45]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                      ERROR               PORTS
yqdbmwna39vb        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 7 seconds ago                                  
iospcjmamp0i        docker-demo.2       mjbright/docker-demo:20   swnode2             Running             Preparing less than a second ago                       
fu96643xoj4b        docker-demo.3       mjbright/docker-demo:20   swnode1             Running             Preparing less than a second ago                       


# rolling-update

Now we will see how we can perform a rolling update.

We initially deployed version 20 of the service, now we will upgrade our whole cluster to version 21


In [46]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                      ERROR               PORTS
yqdbmwna39vb        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 7 seconds ago                                  
iospcjmamp0i        docker-demo.2       mjbright/docker-demo:20   swnode2             Running             Preparing less than a second ago                       
fu96643xoj4b        docker-demo.3       mjbright/docker-demo:20   swnode1             Running             Preparing less than a second ago                       


In [47]:
docker $(docker-machine config swmaster1) service update --image mjbright/docker-demo:21 docker-demo

docker-demo
Since --detach=false was not specified, tasks will be updated in the background.
In a future release, --detach=false will become the default.


In [48]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                      ERROR               PORTS
yqdbmwna39vb        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 12 seconds ago                                 
iospcjmamp0i        docker-demo.2       mjbright/docker-demo:20   swnode2             Running             Running 1 second ago                                   
g3cmw554164z        docker-demo.3       mjbright/docker-demo:21   swnode1             Ready               Preparing less than a second ago                       
fu96643xoj4b         \_ docker-demo.3   mjbright/docker-demo:20   swnode1             Shutdown            Preparing 5 seconds ago                                


Again it may take a few moments until the new image is downloaded and operational on all nodes.

Once the new image has been downloaded and started on all nodes you should see something like:

In [49]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                      ERROR               PORTS
yqdbmwna39vb        docker-demo.1       mjbright/docker-demo:20   swmaster1           Running             Running 13 seconds ago                                 
iospcjmamp0i        docker-demo.2       mjbright/docker-demo:20   swnode2             Running             Running 1 second ago                                   
g3cmw554164z        docker-demo.3       mjbright/docker-demo:21   swnode1             Ready               Preparing less than a second ago                       
fu96643xoj4b         \_ docker-demo.3   mjbright/docker-demo:20   swnode1             Shutdown            Preparing 6 seconds ago                                


### Verifying the service has been updated

Then open your web browser at the page http://localhost:8080 and you should now see a lovely **red** whale.


![](images/docker_red.png)

of if you prefer the command-line ..

In [50]:
NB_docker_loop_until_N_replicas_started docker-demo 3

#NB_pause

2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
2/3
... waiting for 'docker-demo' 3 replicas to start
j6df1lpmiq5e        docker-demo         replicated          3/3                 mjbright/docker-demo:21   *:8080->8080/tcp
Took 95 secs [0h01m35]


In [51]:
docker $(docker-machine config swmaster1) service ls

ID                  NAME                MODE                REPLICAS            IMAGE                     PORTS
j6df1lpmiq5e        docker-demo         replicated          3/3                 mjbright/docker-demo:21   *:8080->8080/tcp


In [52]:
NB_docker_loop_until_service_started docker-demo

j6df1lpmiq5e        docker-demo         replicated          3/3                 mjbright/docker-demo:21   *:8080->8080/tcp


In [53]:
wget -O - http://$MASTERIP:8080

--2017-05-17 19:04:42--  http://192.168.99.104:8080/
Connecting to 192.168.99.104:8080... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/txt]
Saving to: ‘STDOUT’

-                       [<=>                 ]       0  --.-KB/s               


                                                .---------.                                          
                                               .///++++/:.                                          
                                               .///+++//:.                                          
                                               .///+++//:.                                          
                             ``````````````````.:///////:.                       `                  
                             .-///////:://+++//::///////-.                      .--.                
                             .:::///:::///+++///:::///:::.                     .:ss+-`              
          

# drain a node

We can drain a node effectively placing it in 'maintenance mode'.

Draining a node means that it no longer has running tasks on it.

In [54]:
docker $(docker-machine config swmaster1) node ls

ID                            HOSTNAME            STATUS              AVAILABILITY        MANAGER STATUS
63n56ml91pleofqteefs3lfyl     swnode2             Ready               Active              
auzbedso37ndv6his2gimxopa *   swmaster1           Ready               Active              Leader
elcdcst9ubchsrj1s3w9ig0i1     swnode1             Ready               Active              


Let's drain swnode1

In [55]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                 ERROR               PORTS
ow1oor8icdnd        docker-demo.1       mjbright/docker-demo:21   swmaster1           Running             Running 6 seconds ago                             
yqdbmwna39vb         \_ docker-demo.1   mjbright/docker-demo:20   swmaster1           Shutdown            Shutdown 37 seconds ago                           
tt9j3oiz6ayv        docker-demo.2       mjbright/docker-demo:21   swnode2             Running             Running 38 seconds ago                            
iospcjmamp0i         \_ docker-demo.2   mjbright/docker-demo:20   swnode2             Shutdown            Shutdown about a minute ago                       
g3cmw554164z        docker-demo.3       mjbright/docker-demo:21   swnode1             Running             Running about a minute ago                        
fu96643xoj4b         \_ docker-demo.3   mjbright/dock

In [56]:
docker $(docker-machine config swmaster1) node update --availability drain swnode1

swnode1


and now we see that all services on swnode1 are shutdown

In [57]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                    ERROR               PORTS
ow1oor8icdnd        docker-demo.1       mjbright/docker-demo:21   swmaster1           Running             Running 8 seconds ago                                
yqdbmwna39vb         \_ docker-demo.1   mjbright/docker-demo:20   swmaster1           Shutdown            Shutdown 38 seconds ago                              
tt9j3oiz6ayv        docker-demo.2       mjbright/docker-demo:21   swnode2             Running             Running 39 seconds ago                               
iospcjmamp0i         \_ docker-demo.2   mjbright/docker-demo:20   swnode2             Shutdown            Shutdown about a minute ago                          
qcy4qgbb7cdd        docker-demo.3       mjbright/docker-demo:21   swnode2             Ready               Ready less than a second ago                         
g3cmw554164z         \_ docker-demo

But as we had request 3 replicas of our service the Swarm mode will take care to start extra instances on other nodes so that we still have 3 replicas of the service running.

So after a few seconds we see that we once again have 3 replicas in the *Running* state, as shown below:

In [58]:
docker $(docker-machine config swmaster1) service ps docker-demo

ID                  NAME                IMAGE                     NODE                DESIRED STATE       CURRENT STATE                    ERROR               PORTS
ow1oor8icdnd        docker-demo.1       mjbright/docker-demo:21   swmaster1           Running             Running 8 seconds ago                                
yqdbmwna39vb         \_ docker-demo.1   mjbright/docker-demo:20   swmaster1           Shutdown            Shutdown 38 seconds ago                              
tt9j3oiz6ayv        docker-demo.2       mjbright/docker-demo:21   swnode2             Running             Running 40 seconds ago                               
iospcjmamp0i         \_ docker-demo.2   mjbright/docker-demo:20   swnode2             Shutdown            Shutdown about a minute ago                          
qcy4qgbb7cdd        docker-demo.3       mjbright/docker-demo:21   swnode2             Ready               Ready less than a second ago                         
g3cmw554164z         \_ docker-demo

# remove a service

Now let's cleanup by removing our service

In [59]:
docker $(docker-machine config swmaster1) service rm docker-demo

docker-demo


We can check that the service is no longer running:

In [60]:
docker $(docker-machine config swmaster1) service ps docker-demo

NB_continue

no such services: docker-demo


In [61]:
docker $(docker-machine config swmaster1) ps

CONTAINER ID        IMAGE                     COMMAND                  CREATED             STATUS              PORTS               NAMES
f87655b58012        mjbright/docker-demo:21   "/bin/sh -c 'pytho..."   11 seconds ago      Up 10 seconds                           docker-demo.1.ow1oor8icdnd0ygp8guqeo9pi


# Deploying a stack

Since Docker 1.12 there is the concept of a Docker Stack.

Docker Stack, similar to Docker-compose allows to deploy a whole application made up of a group of components.  Stacks extend this concept providing more features such as scaling and node placement.

Let's deploy a stack

First let's clone Alex Ellis' FaaS (Function as a Service - a Docker implementation of "Serverless Computing").

https://github.com/alexellis/faas

There is a quick test drive document here:
https://github.com/alexellis/faas/blob/master/TestDrive.md

from which the following steps were taken:

In [ ]:
git clone https://github.com/alexellis/faas alexellis.faas
cd alexellis.faas

Take the time to look at the docker-compose.yml file.

Note that this is the latest version 3 with extra stack capabilities such as deploy/placement tags.

In [63]:
cat docker-compose.yml

version: "3"
services:

# Core API services are pinned, HA is provided for functions.
    gateway:
        volumes:
            - "/var/run/docker.sock:/var/run/docker.sock"
        ports:
            - 8080:8080
        image: functions/gateway:0.5.5
        networks:
            - functions
        environment:
            dnsrr: "true"  # Temporarily use dnsrr in place of VIP while issue persists on PWD
        deploy:
            placement:
                constraints: [node.role == manager]
 
    prometheus:
        image: functions/prometheus:1.5.2 # Mirrors version 1.5.2 of quay.io/prometheus/prometheus:latest
        volumes:
            - ./prometheus/prometheus.yml:/etc/prometheus/prometheus.yml
            - ./prometheus/alert.rules:/etc/prometheus/alert.rules

        command: "-config.file=/etc/prometheus/prometheus.yml -storage.local.path=/prometheus -storage.local.memory-chunks=10000 --alertmanager.url=http://alertmanager:9093"
        ports:
            - 9090:9090
    

In [64]:
cat deploy_stack.sh

#!/bin/sh

echo "Deploying stack"
docker stack deploy func --compose-file docker-compose.yml



**Note: if running on play-with-docker.com**, copy-paste the following directly

```
      docker swarm init --advertise-addr eth0 && \
        git clone https://github.com/alexellis/faas && \
        cd faas && \
        ./deploy_stack.sh && \
        docker service ls
```

**Note**: Running under play-with-docker.com you will have access to Prometheus also

In [65]:
eval $(docker-machine env swmaster1)

time ./deploy_stack.sh

Deploying stack
Creating network func_functions
Creating service func_webhookstash
Creating service func_echoit
Creating service func_decodebase64
Creating service func_gateway
Creating service func_base64
Creating service func_prometheus
Creating service func_wordcount
Creating service func_markdown
Creating service func_alertmanager
Creating service func_nodeinfo
Creating service func_hubstats

real	0m26.382s
user	0m0.139s
sys	0m0.010s


If you are running on PWD, click on the link to port 8080 at the top of the page.

If you are running via docker-machine open an ssh tunnel to forward this port locally
    docker-machine ssh swmaster1 -L 8080:localhost:8080

### Connect to FaaS dashboard
Then connect to http://localhost:8080 to see the FaaS dashboard

### Inspect your FaaS implementation

From the command-line inspect the stack you've just deployed using
- docker service ls
- docker service ps &lt;service-name&gt;

### Play with FaaS
The point of this exercise was to demonstrate the deploying of the stack, but why not play with FaaS a bit now, you deserve it !

But don't spend too long, you've a Docker-Python class to do next !!

## Visualizing the stack

We can use Mano Marks' visualizer application to view our deployed stack.

Let's launch the visualizer on swmaster1, exposed on port 7070

In [ ]:
docker $(docker-machine config swmaster1) run -it -d -p 7070:8080 -v /var/run/docker.sock:/var/run/docker.sock dockersamples/visualizer

we can now connect to the stack at swmaster1:7070

In [13]:
MASTERIP=$(docker-machine ip swmaster1)

echo "Connect to http://$MASTERIP:7070"

Connect to http://192.168.99.104:7070


You should see something like this

![](images/visualizer.png)

In [11]:
docker $(docker-machine config swmaster1) ps

CONTAINER ID        IMAGE                                   COMMAND                CREATED              STATUS                       PORTS                    NAMES
f77311108ad7        dockersamples/visualizer                "npm start"            About a minute ago   Up About a minute            0.0.0.0:7070->8080/tcp   distracted_torvalds
b4f9c27a7e3e        alexellis2/faas-dockerhubstats:latest   "/usr/bin/fwatchdog"   About an hour ago    Up About an hour                                      func_hubstats.1.4tkp05zwc09tc5pi1fxn9p5cu
baed0d33fa58        functions/alpine:health                 "fwatchdog"            About an hour ago    Up About an hour (healthy)                            func_base64.1.uxs77dkbigzhu4djpps3s14uc
6448893b69ff        functions/gateway:0.5.5                 "./gateway"            About an hour ago    Up About an hour             8080/tcp                 func_gateway.1.kz56s05epbid8k3yi6m0z17wf
ad508f86583a        functions/webhookstash:latest           "

In [66]:
NB_time_taken

Started at Wed May 17 19:05:14 CEST 2017   [1495040345]
Ended   at Wed May 17 19:05:14 CEST 2017   [1495040714]
Took 369 secs [0h06m09]
